# RK Interpolation

This document includes Python codes that conduct Regression Kriging (RK) Interpolation on each waterbody, including Guana Tolomato Matanzas (GTM), Estero Bay (EB), Charlotte Harbor (CH), Biscayne Bay (BB), Big Bend Seagrasses (BBS).
parameters, including Dissolved oxygen (DO_mgl), Salinity (Sal_ppt), Turbidity (Turb_ntu), Temperature (T_c), Secchi (Secc_m), Total Nitrogen (TN_mgl) in arcpy environment.

The analysis is conducted in the separate managed parameters of Total Nitrogen (TN_mgl), Dissolved oxygen (DO_mgl), Salinity (Sal_ppt), Turbidity (Turb_ntu), Temperature (T_c), and Secchi (Secc_m) in arcpy environment.

* [1. Data Preprocess](#preprocessing)
* [2. Generate Shapefiles](#create_shp)
* [3. Regression Kriging for All Stations](#rk_all)

## Load packages

In [33]:
import pandas as pd
import numpy  as np
import arcpy
from arcpy.sa import *
import os, time, math, importlib, sys
path = r'../git/misc'
sys.path.insert(0, path)
import idw_rk
# !install conda install conda-forge::pyproj
import pyproj,csv

importlib.reload(idw_rk)

import warnings
warnings.filterwarnings('ignore')

# 1. Preprocessing <a class="anchor" id="preprocessing"></a>

## 1.1 Load csv files

In [2]:
gis_path = r'E:/Projects/SEACAR_WQ_2024/GIS_Data/'

dfDis = pd.read_csv(gis_path + 'OEAT_Continuous_WQ-2024-Feb-21.csv', low_memory=False)
dfCon = pd.read_csv(gis_path + 'OEAT_Discrete_WQ-2024-Feb-15.csv', low_memory=False)

dfAll = pd.concat([dfDis, dfCon], ignore_index=True)

## 1.2 Select data from 9 am to 17 pm in a day

In [3]:
# Convert string to datetime
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate'], format='%Y-%m-%d %H:%M:%S.%f')

# Include date from 9:00 am to 17:00 pm
start_time = '09:00'
end_time = '17:00'

dfAllTime = dfAll[dfAll['SampleDate'].dt.time.between(pd.to_datetime(start_time).time(), pd.to_datetime(end_time).time())]
dfAllTime.head()

,RowID,ProgramID,ParameterName,ParameterUnits,ProgramLocationID,ActivityType,SampleDate,Year,Month,RelativeDepth,ResultValue,Latitude_DD,Longitude_DD,ManagedAreaName,AreaID,SEACAR_QAQCFlagCode,WaterBody,WbodyAcronym,Season
0,1,512,Water Temperature,Degrees C,CHWW1,NaN,2021-11-25 11:45:00,2021,11,bottom,19.2,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall
1,2,512,Water Temperature,Degrees C,CHWW1,NaN,2021-11-13 16:15:00,2021,11,bottom,22.3,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall
2,3,512,Water Temperature,Degrees C,CHWW1,NaN,2021-10-26 14:15:00,2021,10,bottom,28.3,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall
3,4,512,Water Temperature,Degrees C,CHWW1,NaN,2022-09-16 12:45:00,2022,9,bottom,29.7,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall
4,5,512,Water Temperature,Degrees C,CHWW1,NaN,2022-09-20 10:30:00,2022,9,bottom,29.3,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall


## 1.3 Calculating average values at unique observation points

In [4]:
dfAll_Mean = dfAllTime.groupby(['WaterBody','ParameterName','ParameterUnits', 'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym'])["ResultValue"].agg("mean").reset_index()
dfAll = dfAll_Mean

## 1.4 Convert coordinate system to EPSG: 3086

In [5]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
dfAll[['x', 'y']] = dfAll.apply(transform_coordinates, axis=1)

#### Save aggregated data to csv file

In [6]:
dfAll.to_csv(gis_path + 'OEAT_All_WQ-2024-Feb-21.csv', index=False)

## 2. Prepare for batch interpolation
### 2.1 Preset abbreviation for waterbody and parameter name

In [7]:
area_shortnames = {
    'Guana Tolomato Matanzas': 'GTM',
    'Estero Bay': 'EB',
    'Charlotte Harbor': 'CH',
    'Biscayne Bay': 'BB',
    'Big Bend Seagrasses':'BBS'
}

param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

# Set input parameters
waterbody_names = [
    'Guana Tolomato Matanzas',
    'Estero Bay',
    'Charlotte Harbor',
    'Biscayne Bay',
    'Big Bend Seagrasses'
]

covariates_dict = {
    "GTM":"LDI",
    "EB":"bathymetry+LDI+popden",
    "CH":"bathymetry+LDI+popden+water_flow_wet",
    "BB":"bathymetry+LDI+popden",
    "BBS":"bathymetry+LDI"
}

parameter_names = ['Dissolved Oxygen', 'Salinity', 'Secchi Depth', 'Total Nitrogen', 'Turbidity', 'Water Temperature']
# years = unique_years
seasons = ['Fall', 'Spring', 'Summer', 'Winter']
# shp_folder = gis_path + r"shapefiles_All"
shp_folder = gis_path + r"shapefiles"

### 2.3 Load the table of study periods,  parameters, and seasons

In [8]:
seasons_all = pd.read_csv(gis_path + 'Seasons_all.csv', low_memory=False)

### 2.4 Define output folders


In [9]:
shpAll_folder = gis_path + r"shapefiles_All" 
idwAll_folder = gis_path + r"idw_All"

# Preview dataset
dfAll

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue,x,y
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.849359,480883.256914,587084.624223
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.660736,435816.462864,645279.548973
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.408284,489729.880220,566493.097579
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.454961,480883.256914,587084.624223
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.590892,435816.462864,645279.548973
...,...,...,...,...,...,...,...,...,...,...,...
25196,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.025360,-81.370918,GTM,29.150000,653237.586095,671395.945419
25197,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.026440,-81.369403,GTM,29.500000,653380.952596,671518.961956
25198,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.033611,-81.353027,GTM,29.766667,654940.976043,672348.548670
25199,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.050338,-81.371008,GTM,29.675000,653169.830710,674167.856772


### 2.6 Fill NaN RowID with unique ID, IDW function needs unique ID <a class="anchor" id="reg_id"></a>#

In [10]:
idw_rk.fill_nan_rowids(dfAll, 'RowID')

# Keep RowID as integer
dfAll['RowID'] = dfAll['RowID'].astype(int)
dfAll

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue,x,y,RowID
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.849359,480883.256914,587084.624223,1
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.660736,435816.462864,645279.548973,2
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.408284,489729.880220,566493.097579,3
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.454961,480883.256914,587084.624223,4
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.590892,435816.462864,645279.548973,5
...,...,...,...,...,...,...,...,...,...,...,...,...
25196,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.025360,-81.370918,GTM,29.150000,653237.586095,671395.945419,25197
25197,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.026440,-81.369403,GTM,29.500000,653380.952596,671518.961956,25198
25198,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.033611,-81.353027,GTM,29.766667,654940.976043,672348.548670,25199
25199,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.050338,-81.371008,GTM,29.675000,653169.830710,674167.856772,25200


# 3. Create Shapefiles <a class="anchor" id="reg_create_shp"></a>

In [ ]:
# Empty the shapefile folder
idw_rk.delete_all_files(shpAll_folder)

In [47]:
# Merge interested with latitude and longitude columns
seasons_all_coord = idw_rk.merge_with_lat_long(seasons_all, dfAll)

Number of data rows for BBS, DO_mgl, 2020, Fall: 26
Shapefile for BBS: DO_mgl for year 2020 and season Fall has been saved as SHP_BBS_DO_mgl_2020_Fall.shp
Number of data rows for BBS, Sal_ppt, 2020, Fall: 21
Shapefile for BBS: Sal_ppt for year 2020 and season Fall has been saved as SHP_BBS_Sal_ppt_2020_Fall.shp
Number of data rows for BBS, Secc_m, 2020, Fall: 27
Shapefile for BBS: Secc_m for year 2020 and season Fall has been saved as SHP_BBS_Secc_m_2020_Fall.shp
Number of data rows for BBS, TN_mgl, 2020, Fall: 23
Shapefile for BBS: TN_mgl for year 2020 and season Fall has been saved as SHP_BBS_TN_mgl_2020_Fall.shp
Number of data rows for BBS, Turb_ntu, 2020, Fall: 26
Shapefile for BBS: Turb_ntu for year 2020 and season Fall has been saved as SHP_BBS_Turb_ntu_2020_Fall.shp
Number of data rows for BBS, T_c, 2020, Fall: 26
Shapefile for BBS: T_c for year 2020 and season Fall has been saved as SHP_BBS_T_c_2020_Fall.shp
Number of data rows for BBS, DO_mgl, 2020, Summer: 26
Shapefile for BB

ExecuteError: ERROR 000464: Cannot get exclusive schema lock.  Either being edited or in use by another application or service.
Failed to execute (AddSpatialIndex).


Error occurred: ERROR 000582: Error occurred during execution.

Number of data rows for BBS, T_c, 2021, Summer: 37


ExecuteError: ERROR 000258: Output in_memory\table_merged_intermediate already exists
Failed to execute (CreateFeatureclass).


Error occurred: ERROR 000582: Error occurred during execution.

Number of data rows for BBS, DO_mgl, 2021, Winter: 39


...

Error occurred: ERROR 000582: Error occurred during execution.

Number of data rows for GTM, Turb_ntu, 2017, Summer: 4


ExecuteError: ERROR 000258: Output in_memory\table_merged_intermediate already exists
Failed to execute (CreateFeatureclass).


Error occurred: ERROR 000582: Error occurred during execution.

Number of data rows for GTM, T_c, 2017, Summer: 17


ExecuteError: ERROR 000258: Output in_memory\table_merged_intermediate already exists
Failed to execute (CreateFeatureclass).


Error occurred: ERROR 000582: Error occurred during execution.



### Create Shapefiles

In [ ]:
# Clean the shapefile folder if necessary
# RK.delete_all_files(shp_folder)
# Print number of data points in each shapefile
idw_rk.create_shp_season(seasons_all_coord, shpAll_folder)


# 3. Regression Kriging for both continuous and discrete data<a class="anchor" id="rk_all"></a>

## Loop for all parameters


### Clean the output folder

In [18]:
out_raster_floder = gis_path + "raster_output_rk/"
out_ga_folder     = gis_path + "ga_output_rk/"
diagnostic_folder = gis_path + "diagnostic_rk/"
std_error_folder  = gis_path + "std_error_rk/"

# Clean existing files in folders
idw_rk.delete_all_files(out_raster_floder)
idw_rk.delete_all_files(out_ga_folder)
idw_rk.delete_all_files(diagnostic_folder)
idw_rk.delete_all_files(std_error_folder)

In [41]:
# Write the output in a csv file
with open(gis_path+"rk_results_temp.csv", 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the header line
    cols = list(seasons_all.columns)
    cols.append('covariates')
    csv_writer.writerow(cols)
    
    for i in seasons_all.index:
        s_time =time.time() 
        process,rmse,me,count,file_loc = idw_rk.rk_interpolation(method = "rk",
                                           radius = 10000,
                                           folder_path = gis_path,
                                           waterbody = area_shortnames[seasons_all.iloc[i]["WaterBody"]],
                                           parameter = param_shortnames[seasons_all.iloc[i]["Parameter"]],
                                           year      = seasons_all.iloc[i]["Year"],
                                           season    = seasons_all.iloc[i]['Season'],
                                           covariates= covariates_dict[area_shortnames[seasons_all.iloc[i]["WaterBody"]]],
                                           out_raster_folder = out_raster_floder,
                                           out_ga_folder     = out_ga_folder,
                                           std_error_folder  = std_error_folder,                  
                                           diagnostic_folder = diagnostic_folder)
        e_time =time.time()

        print(f"{int(e_time-s_time)} seconds elapsed for processing {count} points in {i}th row: RMSE: {rmse}, ME: {me}, file exported to {file_loc}")
        csv_writer.writerow([seasons_all.iloc[i]["WaterBody"], 
                             seasons_all.iloc[i]["Year"],
                             seasons_all.iloc[i]['Season'],
                             seasons_all.iloc[i]["Parameter"],
                             file_loc, count, rmse, me,
                             covariates_dict[area_shortnames[seasons_all.iloc[i]["WaterBody"]]]])
        if i%10 == 0: csvfile.flush() # flush the csv file in every 20 rows.
#         seasons_all['RMSE'][i:i+1] = rmse
#         seasons_all['ME'][i:i+1] = me
#         seasons_all['NumDataPoints'][i:i+1] = count
#         seasons_all['Filename'][i:i+1] = file_loc
#     seasons_all.to_csv(gis_path+"result_RK_all.csv")

No data for RK interpolation in SHP_GTM_TN_mgl_2015_Fall.shp, skipping
0 seconds elapsed for processing 0 points in 0th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_GTM_TN_mgl_2015_Winter.shp, skipping
0 seconds elapsed for processing 0 points in 1th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_GTM_TN_mgl_2016_Spring.shp, skipping
0 seconds elapsed for processing 0 points in 2th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_GTM_TN_mgl_2016_Summer.shp, skipping
0 seconds elapsed for processing 0 points in 3th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_GTM_TN_mgl_2016_Fall.shp, skipping
0 seconds elapsed for processing 0 points in 4th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_GTM_TN_mgl_2016_Winter.shp, skipping
0 seconds elapsed for processing 0 points in 5th row: RMSE: nan, ME: nan, file exported to 

Processing file: SHP_BBS_TN_mgl_2021_Fall.shp
--- Time lapse: 603.2613809108734 seconds ---
603 seconds elapsed for processing 33 points in 37th row: RMSE: -1.79769313486e+308, ME: -1.79769313486e+308, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_TN_mgl_2021_Fall_RK.tif
Processing file: SHP_BBS_TN_mgl_2021_Winter.shp
--- Time lapse: 628.4365539550781 seconds ---
628 seconds elapsed for processing 31 points in 38th row: RMSE: -1.79769313486e+308, ME: -1.79769313486e+308, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_TN_mgl_2021_Winter_RK.tif
Processing file: SHP_BBS_TN_mgl_2022_Spring.shp
--- Time lapse: 299.52361488342285 seconds ---
299 seconds elapsed for processing 25 points in 39th row: RMSE: -1.79769313486e+308, ME: -1.79769313486e+308, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_TN_mgl_2022_Spring_RK.tif
Sal_ppt in 2015 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Fail

Processing file: SHP_BB_Sal_ppt_2022_Winter.shp
--- Time lapse: 260.18177366256714 seconds ---
260 seconds elapsed for processing 50 points in 70th row: RMSE: 3.72300308158, ME: 0.224682961631, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BB_Sal_ppt_2022_Winter_RK.tif
Processing file: SHP_BB_Sal_ppt_2023_Spring.shp
--- Time lapse: 165.40335941314697 seconds ---
165 seconds elapsed for processing 48 points in 71th row: RMSE: 3.34391301569, ME: 0.248323914762, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BB_Sal_ppt_2023_Spring_RK.tif
Processing file: SHP_BBS_Sal_ppt_2020_Summer.shp
--- Time lapse: 314.897173166275 seconds ---
315 seconds elapsed for processing 20 points in 72th row: RMSE: -1.79769313486e+308, ME: -1.79769313486e+308, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_Sal_ppt_2020_Summer_RK.tif
Processing file: SHP_BBS_Sal_ppt_2020_Fall.shp
--- Time lapse: 322.89896535873413 seconds ---
323 seconds e

DO_mgl in 2017 Winter caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

4 seconds elapsed for processing 3 points in 102th row: RMSE: nan, ME: nan, file exported to nan
Processing file: SHP_CH_DO_mgl_2018_Spring.shp
--- Time lapse: 699.4871385097504 seconds ---
699 seconds elapsed for processing 36 points in 103th row: RMSE: 0.698904322061, ME: -0.0640475843972, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/CH_DO_mgl_2018_Spring_RK.tif
Processing file: SHP_BB_DO_mgl_2021_Summer.shp
--- Time lapse: 432.6069059371948 seconds ---
432 seconds elapsed for processing 83 points in 104th row: RMSE: 1.08321746126, ME: 0.000895091565736, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BB_DO_mgl_2021_Summer_RK.tif
Processing file: SHP_BB_DO_mgl_2021_Fall.shp
--- Time lapse: 449.78805708885193 seconds ---
449 seconds elapsed for processing 83 points in 105th row: RMSE: 0.815418489492, ME: -0

Turb_ntu in 2017 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 seconds elapsed for processing 3 points in 133th row: RMSE: nan, ME: nan, file exported to nan
Turb_ntu in 2017 Winter caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 seconds elapsed for processing 3 points in 134th row: RMSE: nan, ME: nan, file exported to nan
Processing file: SHP_EB_Turb_ntu_2018_Spring.shp
--- Time lapse: 78.79204177856445 seconds ---
78 seconds elapsed for processing 30 points in 135th row: RMSE: 1.49053900674, ME: 0.0388944796704, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/EB_Turb_ntu_2018_Spring_RK.tif
Turb_ntu in 2016 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

5 seconds elapsed for processing 3 points in 136th row: RMSE: nan, ME: nan, file exported to nan
Turb_nt

Secc_m in 2017 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

1 seconds elapsed for processing 9 points in 167th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_Secc_m_2016_Summer.shp, skipping
0 seconds elapsed for processing 0 points in 168th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_Secc_m_2016_Fall.shp, skipping
0 seconds elapsed for processing 0 points in 169th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_Secc_m_2016_Winter.shp, skipping
0 seconds elapsed for processing 0 points in 170th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_Secc_m_2017_Spring.shp, skipping
0 seconds elapsed for processing 0 points in 171th row: RMSE: nan, ME: nan, file exported to nan
No data for RK interpolation in SHP_EB_Secc_m_2017_Summer.shp, skipping
0 seconds elapsed for p

T_c in 2017 Spring caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

1 seconds elapsed for processing 8 points in 206th row: RMSE: nan, ME: nan, file exported to nan
Processing file: SHP_GTM_T_c_2017_Summer.shp
--- Time lapse: 77.10008430480957 seconds ---
77 seconds elapsed for processing 17 points in 207th row: RMSE: 0.384421323149, ME: -0.0641628659705, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/GTM_T_c_2017_Summer_RK.tif
T_c in 2016 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 seconds elapsed for processing 3 points in 208th row: RMSE: nan, ME: nan, file exported to nan
T_c in 2016 Fall caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

0 seconds elapsed for processing 3 points in 209th row: RMSE: nan, ME: nan, file exported to nan
T_c in 2016 Winter caused an

Processing file: SHP_BBS_T_c_2021_Winter.shp
--- Time lapse: 588.70672082901 seconds ---
588 seconds elapsed for processing 39 points in 238th row: RMSE: -1.79769313486e+308, ME: -1.79769313486e+308, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_T_c_2021_Winter_RK.tif
Processing file: SHP_BBS_T_c_2022_Spring.shp
--- Time lapse: 383.62517523765564 seconds ---
383 seconds elapsed for processing 27 points in 239th row: RMSE: -1.79769313486e+308, ME: -1.79769313486e+308, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_T_c_2022_Spring_RK.tif


### Select the abnormal value from the generated results, here we defined that if the RMSE is too low (<-10000), it is the abnormal value.

In [212]:
error_row = pd.read_csv(gis_path+"rk_results_temp.csv")
error_row_new = error_row.loc[error_row["RMSE"]<-10000]
#error_row_new = error_row_new.drop("Unnamed: 0",axis=1)

error_row_new.to_csv(gis_path+"temp_error.csv")

### Reset the smooth_radius from 10 km to 50 km

In [213]:
with open(gis_path+"temp_error.csv", 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    cols = list(error_row_new.columns)
    cols.append('covariates')
    csv_writer.writerow(cols)
    
    for i in error_row_new.index:
#     for i in range(46,48):
        s_time =time.time() 
        process,rmse,me,count,file_loc = idw_rk.rk_interpolation(method = "rk",
                                           radius = 50000,
                                           folder_path = gis_path,
                                           waterbody = area_shortnames[error_row_new.loc[i]["WaterBody"]],
                                           parameter = param_shortnames[error_row_new.loc[i]["Parameter"]],
                                           year      = error_row_new.loc[i]["Year"],
                                           season    = error_row_new.loc[i]['Season'],
                                           covariates= covariates_dict[area_shortnames[error_row_new.loc[i]["WaterBody"]]],
                                           out_raster_folder = out_raster_floder,
                                           out_ga_folder     = out_ga_folder,
                                           std_error_folder  = std_error_folder,
                                           diagnostic_folder = diagnostic_folder)
        e_time =time.time()

        print(f"{int(e_time-s_time)} seconds elapsed for processing {count} points in {i}th row: RMSE: {rmse}, ME: {me}, file exported to {file_loc}")
        csv_writer.writerow([error_row_new.loc[i]["WaterBody"], 
                             error_row_new.loc[i]["Year"],
                             error_row_new.loc[i]['Season'],
                             error_row_new.loc[i]["Parameter"],
                             file_loc, count, rmse, me,
                             covariates_dict[area_shortnames[error_row_new.loc[i]["WaterBody"]]]])
        if i%10 == 0: csvfile.flush()

Processing file: SHP_GTM_TN_mgl_2017_Summer.shp
--- Time lapse: 86.34440326690674 seconds ---
86 seconds elapsed for processing 13 points in 7th row: RMSE: 0.0742146144189, ME: 0.0237795002935, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/GTM_TN_mgl_2017_Summer_RK.tif
Processing file: SHP_BBS_TN_mgl_2020_Summer.shp
--- Time lapse: 1163.6941142082214 seconds ---
1163 seconds elapsed for processing 21 points in 32th row: RMSE: 0.356204748986, ME: -0.00931046183787, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_TN_mgl_2020_Summer_RK.tif
Processing file: SHP_BBS_TN_mgl_2020_Fall.shp
--- Time lapse: 1412.7538788318634 seconds ---
1412 seconds elapsed for processing 23 points in 33th row: RMSE: 0.120052525975, ME: -0.00489661564162, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_TN_mgl_2020_Fall_RK.tif
Processing file: SHP_BBS_TN_mgl_2020_Winter.shp
--- Time lapse: 2236.005870580673 seconds ---
2236 seconds elaps

Processing file: SHP_BBS_Turb_ntu_2020_Winter.shp
--- Time lapse: 2337.7900857925415 seconds ---
2337 seconds elapsed for processing 35 points in 154th row: RMSE: 3.49646477114, ME: -0.0904679529848, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_Turb_ntu_2020_Winter_RK.tif
Processing file: SHP_BBS_Turb_ntu_2021_Spring.shp
--- Time lapse: 2487.679800271988 seconds ---
2487 seconds elapsed for processing 37 points in 155th row: RMSE: 2.53882626795, ME: -0.220944395334, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_Turb_ntu_2021_Spring_RK.tif
Processing file: SHP_BBS_Turb_ntu_2021_Summer.shp
--- Time lapse: 1270.7053844928741 seconds ---
1270 seconds elapsed for processing 30 points in 156th row: RMSE: 4.59398570716, ME: 0.210471951059, file exported to E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_output_rk/BBS_Turb_ntu_2021_Summer_RK.tif
Processing file: SHP_BBS_Turb_ntu_2021_Fall.shp
--- Time lapse: 2185.7848439216614 seconds ---
218

### Merge the revised results into original results

In [221]:
# Specify the file paths of the CSV files you want to remove
file1 = gis_path+"rk_results_temp.csv"
file2 = gis_path+"temp_error.csv"

original_file = pd.read_csv(file1)
error_row_file= pd.read_csv(file2)

In [222]:
error_index = original_file[original_file["RMSE"]<-10000].index

abnormal_drop = original_file.drop(index=error_index)

In [223]:
new_file = pd.concat([abnormal_drop,error_row_file])
new_file = new_file.drop(columns=['covariates.1'])

### Export the final results

In [224]:
new_file.to_csv(gis_path+"rk_results.csv")

#os.remove(file1)
#os.remove(file2)

### Join IDW all data and IDW of continuous data

In [225]:
result_folder = "E:/Projects/SEACAR_WQ_2024/result/result_v2/"
df_idw_all = pd.read_csv(result_folder + "idw_all.csv") # read results of IDW of all samples
df_idw_con = pd.read_csv(gis_path + "idw_Con/updated_idw.csv") # read results of IDW of continuous samples

# join idw_con with idw_all
df_idw = pd.merge(df_idw_all, df_idw_con, on=['WaterBody', 'Year', 'Season', 'Parameter'], how='left', suffixes=('_idw_all','_idw_con'))


# rename and remove certain columns
df_idw = df_idw.drop(columns = ['Filename_idw_con']).rename(columns={'Filename_idw_all':'Filename',
                                                                     'NumDataPoints_idw_all':'NumDataPoints_all',
                                                                    'NumDataPoints_idw_con':'NumDataPoints_con'})


# convert number of data points to integer and remove NaN
df_idw['NumDataPoints_con'] = df_idw['NumDataPoints_con'].fillna(0).astype(int)

### Join IDW and RK results

In [226]:
df_all.columns

Index(['WaterBody', 'Year', 'Season', 'Parameter', 'Filename',
       'NumDataPoints_all', 'RMSE_idw_all', 'ME_idw_all', 'NumDataPoints_con',
       'RMSE_idw_con', 'ME_idw_con', 'RMSE_rk', 'ME_rk', 'covariates'],
      dtype='object')

In [227]:
df_all = pd.merge(df_idw, new_file, on=['WaterBody', 'Year', 'Season', 'Parameter'], how='left', suffixes=('','_rk'))
df_all = df_all.drop(columns = ['Filename_rk','NumDataPoints']).rename(columns={'Filename_idw':'Filename',
                                                                                   'NumDataPoints_idw':'NumDataPoints',
                                                                                  'RMSE':'RMSE_rk',
                                                                                  'ME':'ME_rk'})
df_all.to_csv(result_folder+"rk_idw_comp.csv")

df_all

,WaterBody,Year,Season,Parameter,Filename,NumDataPoints_all,RMSE_idw_all,ME_idw_all,NumDataPoints_con,RMSE_idw_con,ME_idw_con,RMSE_rk,ME_rk,covariates
0,Guana Tolomato Matanzas,2015,Fall,Total Nitrogen,NoData,0,NaN,NaN,0,NaN,NaN,NaN,NaN,LDI
1,Guana Tolomato Matanzas,2015,Winter,Total Nitrogen,NoData,0,NaN,NaN,0,NaN,NaN,NaN,NaN,LDI
2,Guana Tolomato Matanzas,2016,Spring,Total Nitrogen,NoData,0,NaN,NaN,0,NaN,NaN,NaN,NaN,LDI
3,Guana Tolomato Matanzas,2016,Summer,Total Nitrogen,NoData,0,NaN,NaN,0,NaN,NaN,NaN,NaN,LDI
4,Guana Tolomato Matanzas,2016,Fall,Total Nitrogen,NoData,0,NaN,NaN,0,NaN,NaN,NaN,NaN,LDI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Big Bend Seagrasses,2021,Spring,Water Temperature,SHP_BBS_T_c_2021_Spring.shp,44,2.385210,-0.597934,1,NaN,NaN,1.776868,-0.024218,bathymetry+LDI
236,Big Bend Seagrasses,2021,Summer,Water Temperature,SHP_BBS_T_c_2021_Summer.shp,37,1.289297,-0.228349,1,NaN,NaN,0.802154,-0.044503,bathymetry+LDI
237,Big Bend Seagrasses,2021,Fall,Water Temperature,SHP_BBS_T_c_2021_Fall.shp,38,4.125075,0.342125,1,NaN,NaN,2.905899,-0.086741,bathymetry+LDI
238,Big Bend Seagrasses,2021,Winter,Water Temperature,SHP_BBS_T_c_2021_Winter.shp,39,2.016770,0.319845,1,NaN,NaN,1.475670,0.159890,bathymetry+LDI
